## Segmenting and Clustering Neighborhoods in Toronto

Importing the libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
!pip install geopy
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported.')

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - folium=0.5.0


The following NEW packages will be INSTALLED:

  altair             conda-forge/noarch::altair-4.1.0-py_1
  branca             conda-forge/noarch::branca-0.4.1-py_0
  folium             conda-forge/noarch::folium-0.5.0-py_0
  python_abi         conda-forge/win-64::python_abi-3.7-1_cp37m
  vincent            conda-forge/noarch::vincent-0.4.4-py_1

The following packages will be UPDATED:

  conda                       pkgs/main::conda-4.8.2-py37_0 --> conda-forge::conda-4.8.3-py37hc8dfbb8_1

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi                                         pkgs/main --> conda-forge


Preparing transaction: ...working... done
Verifying transaction: ...working... failed




==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



EnvironmentNotWritableError: The current user does not have write permissions to the target environment.
  environment location: C:\ProgramData\Anaconda3




Libraries imported.


In [2]:
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

Using BeautifulSoup4 package to scrape the data

In [3]:
!pip install BeautifulSoup4
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'html.parser')

In [4]:
Table = soup.find('table', {'class':'wikitable sortable'})

Creating a pandas dataframe from the table craped

In [5]:
import pandas as pd

In [6]:
df = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood'])

In [7]:
count = 0

for tr in Table.find_all('tr'):
    count += 1

print(count)

181


Scraping each row and then scraping each definition, storing them in a list and adding them as rows to the dataframe

In [8]:
i = 0

for tr in Table.find_all('tr'):
    li = []
    for td in tr.find_all('td'):
        li.append(td.text[0:-1])
    if i > 0 :
        df.loc[i] = li[0:3]
    i += 1

In [10]:
df.head(10)

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
10,M1B,Scarborough,"Malvern, Rouge"


Ignoring rows where Borough is set as 'Not assigned'

In [11]:
idxs = df[df['Borough'] == 'Not assigned'].index
df.drop(idxs, inplace = True)

Setting the Neighborhood same as the Borough where Neighborhood is set to 'Not assigned'

In [12]:
df.loc[df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df['Borough']

In [17]:
df.reset_index(inplace = True)
df.head(20)

,index,PostalCode,Borough,Neighborhood
0,3,M3A,North York,Parkwoods
1,4,M4A,North York,Victoria Village
2,5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,6,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,9,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,10,M1B,Scarborough,"Malvern, Rouge"
7,12,M3B,North York,Don Mills
8,13,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,14,M5B,Downtown Toronto,"Garden District, Ryerson"


In [14]:
df.shape

(103, 3)